In [1]:
# This code cell installs packages on Colab
!pip install pyomo
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

from pyomo.environ import *
from pyomo.dae import *
import numpy as np
from google.colab import files

In [8]:
# 1: this script solves the steady-advection-diffusion-reaction equations using the pyomo.dae

# 2: numerical examples are modified from the paper: Isogeometric analysis: CAD, finite elements, NURBS,
# exact geometry and mesh refinement, T.J.R. Hughes et.al

# 3: The code structure refers to the paper: pyomo.dae: a modeling and automatic discretization
# framework for optimization with differential and algebraic equations, Nicholson et al.


# space-time variables
m      = ConcreteModel()
m.x    = ContinuousSet(bounds = (0,1))
m.y    = ContinuousSet(bounds = (0,1))
m.phi  = Var(m.x, m.y)


# define derivative variables
m.DphiDx   = DerivativeVar(m.phi, wrt = m.x)
m.DphiDy   = DerivativeVar(m.phi, wrt = m.y)
m.DphiDx_2 = DerivativeVar(m.phi, wrt = (m.x, m.x))
m.DphiDy_2 = DerivativeVar(m.phi, wrt = (m.y, m.y))

# define PDE parameters
m.c 	  = Param(initialize = 1.0)                              # the reaction constant
m.nu      = Param(initialize = 1e-6)            # the diffusivity
m.bx      = Param(initialize = 1.0/2)           # the advection velocity, x-component
m.by      = Param(initialize = np.sqrt(3)/2.0)  # the advection velocity, y-component


# define adr pde, i: x, j: y
def ADR_pde(m,i,j):
	if j == m.y.first(): #or j==m.y.last():
	 	return Constraint.Skip
	if i == m.x.first(): #or i==m.x.last():
	 	return Constraint.Skip
	return m.bx * m.DphiDx[i,j] + m.by * m.DphiDy[i,j]- m.nu*m.DphiDx_2[i,j] - m.nu*m.DphiDy_2[i,j]   == 0.0 #+ m.c*m.phi[i,j]
m.pde = Constraint(m.x, m.y, rule = ADR_pde )   

#Bottom boundary condition [phi = 1 @ y = 0]
def BC1(m,i):
	if i == m.x.first(): #or i == m.x.last():
		return Constraint.Skip
	return m.phi[i,m.y.first()] == 1.0
m.BCy_0 = Constraint(m.x, rule = BC1)


# Left boundary condition [phi = 1 @ 0<y<0.2, phi = 0 @ y>0.2]
def BC4(m,j):
	if j> 0.2:
		return m.phi[m.x.first(),j] == 0.0
	else:
		return m.phi[m.x.first(),j] == 1.0
m.BCx_0 = Constraint(m.y, rule = BC4)

# trivial obj
m.obj = Objective(expr = 1)

# discretization and solve
discretizer = TransformationFactory('dae.collocation')
discretizer.apply_to(m,nfe=10,ncp=3,wrt=m.x)
discretizer.apply_to(m,nfe=10,ncp=3,wrt=m.y)

solver = SolverFactory('ipopt')
solver.options['max_iter']= 500 #number of iterations you wish
results = solver.solve(m, tee=True) 

# extract data
x   = np.zeros(len(m.x))
y   = np.zeros(len(m.y))
phi = np.zeros( ( len(m.x), len(m.y) ) )

for i in range(len(m.x)):
	for j in range(len(m.y)):
		x[i] = value(m.x[i+1])
		y[j] = value(m.y[j+1])
		phi[i,j] = value(m.phi[x[i],y[j]])

# save the data
np.savetxt('skewed-x3.csv', x, delimiter=',')   
np.savetxt('skewed-y3.csv', y, delimiter=',') 
np.savetxt('skewed-phi3.csv', phi, delimiter=',')     

Ipopt 3.12.13: max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    22261
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     4681
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds: